In [1]:
import numpy as np

from stochastic_volatility_models.src.core.underlying import Underlying
from stochastic_volatility_models.src.core.volatility_surface import VolatilitySurface
from stochastic_volatility_models.src.core.pricing_models import PricingModel
from stochastic_volatility_models.src.models.heston import HestonModel

2024-07-30 18:49:38.076 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
ticker = "SPX"
spx = Underlying(ticker)
pricing_model = PricingModel("Black-Scholes")
time = np.datetime64("2022-03-03")

In [3]:
expiries = np.array(
	["2022-03-04", "2022-03-09", "2022-03-11", "2022-03-18", "2022-03-23", "2022-03-25", "2022-03-30", "2022-03-31", "2022-04-01", "2022-04-08", "2022-04-14", "2022-04-22", "2022-04-29", "2022-05-20", "2022-05-31", "2022-06-17", "2022-06-30", "2022-07-15", "2022-07-29", "2022-08-31"], dtype=np.datetime64
)

strikes = np.array(
	[
		2200,
		2400,
		2600,
		2800,
		3000,
		3200,
		3400,
		3500,
		3600,
		3700,
		3800,
		3850,
		3900,
		3950,
		3975,
		4000,
		4025,
		4040,
		4050,
		4060,
		4070,
		4075,
		4080,
		4090,
		4100,
		4110,
		4120,
		4125,
		4130,
		4140,
		4150,
		4160,
		4170,
		4175,
		4180,
		4190,
		4200,
		4210,
		4220,
		4225,
		4230,
		4240,
		4250,
		4260,
		4270,
		4275,
		4280,
		4290,
		4300,
		4310,
		4320,
		4325,
		4330,
		4340,
		4350,
		4360,
		4370,
		4375,
		4380,
		4390,
		4400,
		4410,
		4420,
		4425,
		4430,
		4440,
		4450,
		4460,
		4470,
		4475,
		4480,
		4490,
		4500,
		4510,
		4525,
		4550,
		4600,
		4650,
		4700,
		4800,
		5000,
		5200,
		5400,
	]
)
# strikes = np.array([4100, 4200,4250, 4260, 4270, 4280, 4290, 4300, 4310, 4320, 4340, 4350, 4360, 4370, 4380, 4400, 4500])
spx_vs = VolatilitySurface(
	underlying=spx,
	expiries=expiries,
	strikes=strikes,
	monthly=False,
)

In [4]:
model = HestonModel(
	parameters={
		"initial_variance": 0.05,
		"long_term_volatility": 0.05,
		"volatility_of_volatility": 0.3,
		"mean_reversion_rate": 2.0,
		"wiener_correlation": -0.5,
	}
)

In [5]:
from scipy.optimize import minimize as minimise


def sq(x, a, b, c):
	return a * x**2 + b * x + c


minimise(sq, 0, args=(2, 2, 2))

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.5
        x: [-5.000e-01]
      nit: 1
      jac: [ 4.470e-08]
 hess_inv: [[1]]
     nfev: 6
     njev: 3

In [6]:
np.array(list(model.parameters.values()))

array([ 0.05,  0.05,  0.3 ,  2.  , -0.5 ])

In [7]:
model.fit(
	index_volatility_surface=spx_vs,
	volatility_index_volatility_surface=spx_vs,
	time=time,
	pricing_model=PricingModel(),
)

TypeError: minimise_cost_function() missing 1 required positional argument: 'weights'